In [14]:
% load_ext autoreload
% autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import utils
import train
import cvae

In [3]:
train_iter, val_iter, test, DE, EN = utils.torchtext_extract()

downloading de-en.tgz
.data/iwslt/de-en/IWSLT16.TED.tst2013.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2013.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2014.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2014.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2014.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2014.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.tst2013.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2010.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2011.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2011.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TED.tst2012.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.dev2012.de-en.en.xml
.data/iwslt/de-en/IWSLT16.TEDX.dev2012.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2013.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2010.de-en.de.xml
.data/iwslt/de-en/IWSLT16.TED.tst2012.de-en.de.xml
.data/iwslt/de-en/train.tags.de-en.de
.data/iwslt/de-e

In [4]:
model_name = "test"

gpu = True

num_layers = 2
embed_size = 100
hidden_size = 100
latent_size = 20

num_epochs=10

In [6]:
model = cvae.CVAE(len(DE.vocab), len(EN.vocab), embed_size, hidden_size, latent_size, num_layers)
if gpu:
    model.cuda()

/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [22]:
train.train(model, model_name, train_iter, val_iter, DE, EN, num_epochs, gpu)

  0%|          | 0/3722 [00:00<?, ?it/s]/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "
  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)



ValueError: too many values to unpack (expected 2)

In [20]:
import utils
val_perp = utils.perp_bound(model, val_iter, gpu)
print(val_perp)

  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "
100%|██████████| 18/18 [00:00<00:00, 41.77it/s]

226.71776597795792


In [24]:
DE.vocab.stoi['<s>']

0